In [1]:
path = "D:\Cox-nnet\pathway_mask_tcga_brca.csv"

In [2]:
##%%writefile DataLoader.py
import numpy as np
import pandas as pd
import torch

def sort_data(path):
	''' sort the genomic and clinical data w.r.t. survival time (OS_MONTHS) in descending order
	Input:
		path: path to input dataset (which is expected to be a csv file).
	Output:
		x: sorted genomic inputs.
		ytime: sorted survival time (OS_MONTHS) corresponding to 'x'.
		yevent: sorted censoring status (OS_EVENT) corresponding to 'x', where 1 --> deceased; 0 --> censored.
		age: sorted age corresponding to 'x'.
	'''

	data = pd.read_csv(path)

	data.sort_values("OS_MONTHS", ascending = False, inplace = True)

	x = data.drop(["SAMPLE_ID", "OS_MONTHS", "OS_EVENT", "AGE"], axis = 1).values #remove age
	ytime = data.loc[:, ["OS_MONTHS"]].values
	yevent = data.loc[:, ["OS_EVENT"]].values
	age = data.loc[:, ["AGE"]].values   #remove age

	return(x, ytime, yevent, age)  #remove age


def load_data(path, dtype):
	'''Load the sorted data, and then covert it to a Pytorch tensor.
	Input:
		path: path to input dataset (which is expected to be a csv file).
		dtype: define the data type of tensor (i.e. dtype=torch.FloatTensor)
	Output:
		X: a Pytorch tensor of 'x' from sort_data().
		YTIME: a Pytorch tensor of 'ytime' from sort_data().
		YEVENT: a Pytorch tensor of 'yevent' from sort_data().
		AGE: a Pytorch tensor of 'age' from sort_data().
	'''
	x, ytime, yevent, age = sort_data(path)  #remove age
	###

	X = torch.from_numpy(x).type(dtype)
	YTIME = torch.from_numpy(ytime).type(dtype)
	YEVENT = torch.from_numpy(yevent).type(dtype)
	AGE = torch.from_numpy(age).type(dtype)
	###if gpu is being used
	if torch.cuda.is_available():
		X = X.cuda()
		YTIME = YTIME.cuda()
		YEVENT = YEVENT.cuda()
		AGE = AGE.cuda() #remove age
	###
	return(X, YTIME, YEVENT, AGE)   #remove age


def load_pathway(path, dtype):
	'''Load a bi-adjacency matrix of pathways, and then covert it to a Pytorch tensor.
	Input:
		path: path to input dataset (which is expected to be a csv file).
		dtype: define the data type of tensor (i.e. dtype=torch.FloatTensor)
	Output:
		PATHWAY_MASK: a Pytorch tensor of the bi-adjacency matrix of pathways.
	'''
	pathway_mask = pd.read_csv(path, index_col = 0).values

	PATHWAY_MASK = torch.from_numpy(pathway_mask).type(dtype)
	###if gpu is being used
	if torch.cuda.is_available():
		PATHWAY_MASK = PATHWAY_MASK.cuda()
	###
	return(PATHWAY_MASK)


In [3]:
# pwd()

In [4]:
# PATHWAY_MASK=load_pathway(path,dtype)

In [5]:
# PATHWAY_MASK[500]

In [6]:
# x_train, ytime_train, yevent_train, age_train = load_data("train_tcga_ov.csv", dtype)
# x_valid, ytime_valid, yevent_valid, age_valid = load_data("validation_data_tcga_ov.csv", dtype)
# x_test, ytime_test, yevent_test, age_test = load_data("test_final_tcga_Brca.csv", dtype)

In [7]:
import torch
import torch.nn as nn

class Cox_PASNet(nn.Module):
# 	def __init__(self, In_Nodes, Pathway_Nodes, Hidden_Nodes, Out_Nodes, Pathway_Mask):
	def __init__(self, In_Nodes, Hidden_Nodes, Out_Nodes):
		super(Cox_PASNet, self).__init__()
		self.tanh = nn.Tanh()
# 		self.pathway_mask = Pathway_Mask
		###gene layer --> pathway layer
# 		self.sc1 = nn.Linear(In_Nodes, Pathway_Nodes)
		self.sc1 = nn.Linear(In_Nodes,860)
		###pathway layer --> hidden layer
# 		self.sc2 = nn.Linear(Pathway_Nodes, Hidden_Nodes)
		self.sc2 = nn.Linear(860,Hidden_Nodes)
		###hidden layer --> hidden layer 2
		self.sc3 = nn.Linear(Hidden_Nodes, Out_Nodes, bias=False)
		###hidden layer 2 + age --> Cox layer
		self.sc4 = nn.Linear(Out_Nodes+1, 1, bias = False) # replace nn.Linear(Out_Nodes, 1, bias = False)
		self.sc4.weight.data.uniform_(-0.001, 0.001)
		###randomly select a small sub-network
# 		self.do_m1 = torch.ones(Pathway_Nodes)
		self.do_m2 = torch.ones(Hidden_Nodes)
		###if gpu is being used
		if torch.cuda.is_available():
			self.do_m1 = self.do_m1.cuda()
			self.do_m2 = self.do_m2.cuda()
		###

	def forward(self, x_1, x_2):
		###force the connections between gene layer and pathway layer w.r.t. 'pathway_mask'
# 		self.sc1.weight.data = self.sc1.weight.data.mul(self.pathway_mask)
		x_1 = self.tanh(self.sc1(x_1))
		if self.training == True: ###construct a small sub-network for training only
			x_1 = x_1.mul(self.do_m1)
		x_1 = self.tanh(self.sc2(x_1))
		if self.training == True: ###construct a small sub-network for training only
			x_1 = x_1.mul(self.do_m2)
		x_1 = self.tanh(self.sc3(x_1))
		###combine age with hidden layer 2
		x_cat = torch.cat((x_1, x_2), 1) # will have to check variable x_2 and , 1
		lin_pred = self.sc4(x_cat)

		return lin_pred

In [8]:
#%%writefile SubNetwork_SparseCoding.py
import torch
import numpy as np
import math

def dropout_mask(n_node, drop_p):
	'''Construct a binary matrix to randomly drop nodes in a layer.
	Input:
		n_node: number of nodes in the layer.
		drop_p: the probability that a node is to be dropped.
	Output:
		mask: a binary matrix, where 1 --> keep the node; 0 --> drop the node.
	'''
	keep_p = 1.0 - drop_p
	mask = torch.Tensor(np.random.binomial(1, keep_p, size=n_node))
	###if gpu is being used
	if torch.cuda.is_available():
		mask = mask.cuda()
	###
	return mask

def s_mask(sparse_level, param_matrix, nonzero_param_1D, dtype):
	'''Construct a binary matrix w.r.t. a sparsity level of weights between two consecutive layers
	Input:
		sparse_level: a percentage value in [0, 100) represents the proportion of weights in a sub-network to be dropped.
		param_matrix: a weight matrix for entrie network.
		nonzero_param_1D: 1D of non-zero 'param_matrix' (which is the weights selected from a sub-network).
		dtype: define the data type of tensor (i.e. dtype=torch.FloatTensor).
	Output:
		param_mask: a binary matrix, where 1 --> keep the node; 0 --> drop the node.
	'''
	###take the absolute values of param_1D
	non_neg_param_1D = torch.abs(nonzero_param_1D)
	###obtain the number of params
	num_param = nonzero_param_1D.size(0)
	###obtain the kth number based on sparse_level
	top_k = math.ceil(num_param*(100-sparse_level)*0.01)
	###obtain the k largest params
	sorted_non_neg_param_1D, indices = torch.topk(non_neg_param_1D, top_k)
	param_mask = torch.abs(param_matrix) > sorted_non_neg_param_1D.min()
	param_mask = param_mask.type(dtype)
	###if gpu is being used
	if torch.cuda.is_available():
		param_mask = param_mask.cuda()
	###
	return param_mask


In [9]:
#writefile Survival_CostFunc_CIndex.py
import torch

def R_set(x):
	'''Create an indicator matrix of risk sets, where T_j >= T_i.
	Note that the input data have been sorted in descending order.
	Input:
		x: a PyTorch tensor that the number of rows is equal to the number of samples.
	Output:
		indicator_matrix: an indicator matrix (which is a lower traiangular portions of matrix).
	'''
	n_sample = x.size(0)
	matrix_ones = torch.ones(n_sample, n_sample)
	indicator_matrix = torch.tril(matrix_ones)

	return(indicator_matrix)

def neg_par_log_likelihood(pred, ytime, yevent):
	'''Calculate the average Cox negative partial log-likelihood.
	Note that this function requires the input data have been sorted in descending order.
	Input:
		pred: linear predictors from trained model.
		ytime: true survival time from load_data().
		yevent: true censoring status from load_data().
	Output:
		cost: the cost that is to be minimized.
	'''
	n_observed = yevent.sum(0)
	ytime_indicator = R_set(ytime)
	###if gpu is being used
	if torch.cuda.is_available():
		ytime_indicator = ytime_indicator.cuda()
	###
	risk_set_sum = ytime_indicator.mm(torch.exp(pred))
	diff = pred - torch.log(risk_set_sum)
	sum_diff_in_observed = torch.transpose(diff, 0, 1).mm(yevent)
	cost = (- (sum_diff_in_observed / n_observed)).reshape((-1,))

	return(cost)


def c_index(pred, ytime, yevent):
	'''Calculate concordance index to evaluate models.
	Input:
		pred: linear predictors from trained model.
		ytime: true survival time from load_data().
		yevent: true censoring status from load_data().
	Output:
		concordance_index: c-index (between 0 and 1).
	'''
	n_sample = len(ytime)
	ytime_indicator = R_set(ytime)
	ytime_matrix = ytime_indicator - torch.diag(torch.diag(ytime_indicator))
	###T_i is uncensored
	censor_idx = (yevent == 0).nonzero()
	zeros = torch.zeros(n_sample)
	ytime_matrix[censor_idx, :] = zeros
	###1 if pred_i < pred_j; 0.5 if pred_i = pred_j
	pred_matrix = torch.zeros_like(ytime_matrix)
	for j in range(n_sample):
		for i in range(n_sample):
			if pred[i] < pred[j]:
				pred_matrix[j, i]  = 1
			elif pred[i] == pred[j]:
				pred_matrix[j, i] = 0.5

	concord_matrix = pred_matrix.mul(ytime_matrix)
	###numerator
	concord = torch.sum(concord_matrix)
	###denominator
	epsilon = torch.sum(ytime_matrix)
	###c-index = numerator/denominator
	concordance_index = torch.div(concord, epsilon)
	###if gpu is being used
	if torch.cuda.is_available():
		concordance_index = concordance_index.cuda()
	###
	return(concordance_index)






In [10]:
# from Model import Cox_PASNet
# from SubNetwork_SparseCoding import dropout_mask, s_mask
# from Survival_CostFunc_CIndex import R_set, neg_par_log_likelihood, c_index

import torch
import torch.optim as optim
import copy
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
dtype = torch.FloatTensor
# Dropout_Rate = [0.7,0.5]

def trainCoxPASNet(train_x, train_age, train_ytime, train_yevent,eval_x, eval_age, eval_ytime, eval_yevent,In_Nodes, Hidden_Nodes, Out_Nodes,Learning_Rate, L2, Num_Epochs, Dropout_Rate):


	net = Cox_PASNet(In_Nodes, Hidden_Nodes, Out_Nodes)
	###if gpu is being used
	if torch.cuda.is_available():
		net.cuda()
	###
	###optimizer
	opt = optim.Adam(net.parameters(), lr=Learning_Rate, weight_decay = L2)

	for epoch in range(Num_Epochs+1):
		net.train()
		opt.zero_grad() ###reset gradients to zeros
		###Randomize dropout masks
# 		net.do_m1 = dropout_mask(Dropout_Rate[0])
# 		net.do_m2 = dropout_mask(Hidden_Nodes, Dropout_Rate[1])
		net.do_m1 = dropout_mask(860, Dropout_Rate[0])
		net.do_m2 = dropout_mask(150, Dropout_Rate[1])
# 		print("*******",net.do_m1)


		pred = net(train_x, train_age) ###Forward
		loss = neg_par_log_likelihood(pred, train_ytime, train_yevent) ###calculate loss
		loss.backward() ###calculate gradients
		opt.step() ###update weights and biases

# 		net.sc1.weight.data = net.sc1.weight.data.mul(net.pathway_mask) ###force the connections between gene layer and pathway layer

		###obtain the small sub-network's connections
		do_m1_grad = copy.deepcopy(net.sc2.weight._grad.data)
		do_m2_grad = copy.deepcopy(net.sc3.weight._grad.data)
		do_m1_grad_mask = torch.where(do_m1_grad == 0, do_m1_grad, torch.ones_like(do_m1_grad))
		do_m2_grad_mask = torch.where(do_m2_grad == 0, do_m2_grad, torch.ones_like(do_m2_grad))
		###copy the weights
		net_sc2_weight = copy.deepcopy(net.sc2.weight.data)
		net_sc3_weight = copy.deepcopy(net.sc3.weight.data)

		###serializing net
		net_state_dict = net.state_dict()

		###Sparse Coding
		###make a copy for net, and then optimize sparsity level via copied net
		copy_net = copy.deepcopy(net)
		copy_state_dict = copy_net.state_dict()
		for name, param in copy_state_dict.items():
			###omit the param if it is not a weight matrix
			if not "weight" in name:
				continue
			###omit gene layer
			if "sc1" in name:
				continue
			###stop sparse coding
			if "sc4" in name:
				break
			###sparse coding between the current two consecutive layers is in the trained small sub-network
			if "sc2" in name:
				active_param = net_sc2_weight.mul(do_m1_grad_mask)
			if "sc3" in name:
				active_param = net_sc3_weight.mul(do_m2_grad_mask)
			nonzero_param_1d = active_param[active_param != 0]
			if nonzero_param_1d.size(0) == 0: ###stop sparse coding between the current two consecutive layers if there are no valid weights
				break
			copy_param_1d = copy.deepcopy(nonzero_param_1d)
			###set up potential sparsity level in [0, 100)
			S_set =  torch.arange(100, -1, -1)[1:]
			copy_param = copy.deepcopy(active_param)
			S_loss = []
			for S in S_set:
				param_mask = s_mask(sparse_level = S.item(), param_matrix = copy_param, nonzero_param_1D = copy_param_1d, dtype = dtype)
				transformed_param = copy_param.mul(param_mask)
				copy_state_dict[name].copy_(transformed_param)
				copy_net.train()
				y_tmp = copy_net(train_x, train_age)
				loss_tmp = neg_par_log_likelihood(y_tmp, train_ytime, train_yevent)
				S_loss.append(loss_tmp)
			#print(type(S_loss))
			#print(type(S_set))
			S_loss = torch.tensor(S_loss)
			S_loss = S_loss.detach().numpy()
			S_set = S_set.detach().numpy()
			#print(type(S_loss))
			#print(type(S_set))
			###apply cubic interpolation
			interp_S_loss = interp1d(S_set, S_loss, kind='cubic')
			interp_S_set = torch.linspace(min(S_set), max(S_set), steps=100)
			interp_loss = interp_S_loss(interp_S_set)
			optimal_S = interp_S_set[np.argmin(interp_loss)]
			optimal_param_mask = s_mask(sparse_level = optimal_S.item(), param_matrix = copy_param, nonzero_param_1D = copy_param_1d, dtype = dtype)
			if "sc2" in name:
				final_optimal_param_mask = torch.where(do_m1_grad_mask == 0, torch.ones_like(do_m1_grad_mask), optimal_param_mask)
				optimal_transformed_param = net_sc2_weight.mul(final_optimal_param_mask)
			if "sc3" in name:
				final_optimal_param_mask = torch.where(do_m2_grad_mask == 0, torch.ones_like(do_m2_grad_mask), optimal_param_mask)
				optimal_transformed_param = net_sc3_weight.mul(final_optimal_param_mask)
			###update weights in copied net
			copy_state_dict[name].copy_(optimal_transformed_param)
			###update weights in net
			net_state_dict[name].copy_(optimal_transformed_param)

		if epoch % 200 == 0:
			net.train()
			train_pred = net(train_x, train_age)
			train_loss = neg_par_log_likelihood(train_pred, train_ytime, train_yevent).view(1,)

			net.eval()
			eval_pred = net(eval_x, eval_age)
			eval_loss = neg_par_log_likelihood(eval_pred, eval_ytime, eval_yevent).view(1,)

			train_cindex = c_index(train_pred, train_ytime, train_yevent)
			eval_cindex = c_index(eval_pred, eval_ytime, eval_yevent)
			print("Loss in Train: ", train_loss)

	return (train_loss, eval_loss, train_cindex, eval_cindex)

In [11]:
#from DataLoader import load_data, load_pathway
#from Train import trainCoxPASNet

import torch
import numpy as np


dtype = torch.FloatTensor
''' Net Settings'''
In_Nodes = 270 ###number of genes
# Pathway_Nodes = 860 ###number of pathways
Hidden_Nodes = 150 ###number of hidden nodes
Out_Nodes = 30 ###number of hidden nodes in the last hidden layer
''' Initialize '''
Initial_Learning_Rate = [0.03, 0.01, 0.001, 0.00075]
# L2_Lambda = [0.1]
num_epochs = 100 ###for grid search
Num_EPOCHS =900 ###for training
Dropout_Rate = [0.7, 0.5]
###sub-network setup
L2_Lambda = [0.1, 0.01, 0.005, 0.001]
''' load data and pathway '''
# pathway_mask = load_pathway("pathway_mask_tcga_ov.csv", dtype)

# x_train, ytime_train, yevent_train, age_train = load_data("train_tcga_ov.csv", dtype)
# x_valid, ytime_valid, yevent_valid, age_valid = load_data("validation_data_tcga_ov.csv", dtype)
# x_test, ytime_test, yevent_test, age_test = load_data("test_final_tcga_ov.csv", dtype)
x_train, ytime_train, yevent_train, age_train = load_data("/content/train_tcga_Brca.csv", dtype)
x_valid, ytime_valid, yevent_valid, age_valid = load_data("/content/validation_data_tcga_Brca.csv", dtype)
x_test, ytime_test, yevent_test, age_test = load_data("/content/test_final_tcga_Brca.csv", dtype)

opt_l2_loss = 0
opt_lr_loss = 0
opt_loss = torch.Tensor([float("Inf")])
###if gpu is being used
if torch.cuda.is_available():
	opt_loss = opt_loss.cuda()
###
opt_c_index_va = 0
opt_c_index_tr = 0
###grid search the optimal hyperparameters using train and validation data
for l2 in L2_Lambda:
	for lr in Initial_Learning_Rate:
		loss_train, loss_valid, c_index_tr, c_index_va = trainCoxPASNet(x_train, age_train, ytime_train, yevent_train,x_valid, age_valid, ytime_valid, yevent_valid,In_Nodes, Hidden_Nodes, Out_Nodes,lr, l2, num_epochs, Dropout_Rate)
		if loss_valid < opt_loss:
			opt_l2_loss = l2
			opt_lr_loss = lr
			opt_loss = loss_valid
			opt_c_index_tr = c_index_tr
			opt_c_index_va = c_index_va
		print ("L2: ", l2, "LR: ", lr, "Loss in Validation: ", loss_valid)

###train Cox-PASNet with optimal hyperparameters using train data, and then evaluate the trained model with test data
###Note that test data are only used to evaluate the trained Cox-PASNet
loss_train, loss_test, c_index_tr, c_index_te = trainCoxPASNet(x_train, age_train, ytime_train, yevent_train, \
							x_test, age_test, ytime_test, yevent_test, \
							In_Nodes, Hidden_Nodes, Out_Nodes, \
							opt_lr_loss, opt_l2_loss, Num_EPOCHS, Dropout_Rate)
print ("Optimal L2: ", opt_l2_loss, "Optimal LR: ", opt_lr_loss)
print("C-index in Test: ", c_index_te)

Loss in Train:  tensor([5.0042], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.03 Loss in Validation:  tensor([3.9124], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([5.0460], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.01 Loss in Validation:  tensor([3.8228], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([5.0856], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.001 Loss in Validation:  tensor([3.7989], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([5.0866], grad_fn=<ViewBackward0>)
L2:  0.1 LR:  0.00075 Loss in Validation:  tensor([3.7985], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([5.0041], grad_fn=<ViewBackward0>)
L2:  0.01 LR:  0.03 Loss in Validation:  tensor([3.9132], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([5.0502], grad_fn=<ViewBackward0>)
L2:  0.01 LR:  0.01 Loss in Validation:  tensor([3.8193], grad_fn=<ViewBackward0>)
Loss in Train:  tensor([5.0884], grad_fn=<ViewBackward0>)
L2:  0.01 LR:  0.001 Loss in Validation:  tensor([3.7978], grad_fn=<ViewBackward0>)
Loss in Trai

In [15]:
#%%writefile Train_for_Interpret.py
# from Model import Cox_PASNet
# from SubNetwork_SparseCoding import dropout_mask, s_mask
# from Survival_CostFunc_CIndex import R_set, neg_par_log_likelihood, c_index

import torch
import torch.optim as optim
import copy
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
dtype = torch.FloatTensor

def InterpretCoxPASNet(x, age, ytime, yevent, \
						In_Nodes, Hidden_Nodes, Out_Nodes, \
						Learning_Rate, L2, Num_Epochs, Dropout_Rate, outpath):

	net = Cox_PASNet(In_Nodes, Hidden_Nodes, Out_Nodes)
	###if gpu is being used
	if torch.cuda.is_available():
		net.cuda()
	###
	###optimizer
	opt = optim.Adam(net.parameters(), lr=Learning_Rate, weight_decay = L2)

	for epoch in range(Num_Epochs+1):
		net.train()
		opt.zero_grad() ###reset gradients to zeros
		###Randomize dropout masks
		net.do_m1 = dropout_mask(860, Dropout_Rate[0])
		net.do_m2 = dropout_mask(Hidden_Nodes, Dropout_Rate[1])

		pred = net(x, age) ###Forward
		loss = neg_par_log_likelihood(pred, ytime, yevent) ###calculate loss
		loss.backward() ###calculate gradients
		opt.step() ###update weights and biases

# 		net.sc1.weight.data = net.sc1.weight.data.mul(net.pathway_mask) ###force the connections between gene layer and pathway layer

		###obtain the small sub-network's connections
		do_m1_grad = copy.deepcopy(net.sc2.weight._grad.data)
		do_m2_grad = copy.deepcopy(net.sc3.weight._grad.data)
		do_m1_grad_mask = torch.where(do_m1_grad == 0, do_m1_grad, torch.ones_like(do_m1_grad))
		do_m2_grad_mask = torch.where(do_m2_grad == 0, do_m2_grad, torch.ones_like(do_m2_grad))
		###copy the weights
		net_sc2_weight = copy.deepcopy(net.sc2.weight.data)
		net_sc3_weight = copy.deepcopy(net.sc3.weight.data)

		###serializing net
		net_state_dict = net.state_dict()

		###Sparse Coding
		###make a copy for net, and then optimize sparsity level via copied net
		copy_net = copy.deepcopy(net)
		copy_state_dict = copy_net.state_dict()
		for name, param in copy_state_dict.items():
			###omit the param if it is not a weight matrix
			if not "weight" in name:
				continue
			###omit gene layer
			if "sc1" in name:
				continue
			###stop sparse coding
			if "sc4" in name:
				break
			###sparse coding between the current two consecutive layers is in the trained small sub-network
			if "sc2" in name:
				active_param = net_sc2_weight.mul(do_m1_grad_mask)
			if "sc3" in name:
				active_param = net_sc3_weight.mul(do_m2_grad_mask)
			nonzero_param_1d = active_param[active_param != 0]
			if nonzero_param_1d.size(0) == 0: ###stop sparse coding between the current two consecutive layers if there are no valid weights
				break
			copy_param_1d = copy.deepcopy(nonzero_param_1d)
			###set up potential sparsity level in [0, 100)
			S_set =  torch.arange(100, -1, -1)[1:]
			copy_param = copy.deepcopy(active_param)
			S_loss = []
			for S in S_set:
				param_mask = s_mask(sparse_level = S.item(), param_matrix = copy_param, nonzero_param_1D = copy_param_1d, dtype = dtype)
				transformed_param = copy_param.mul(param_mask)
				copy_state_dict[name].copy_(transformed_param)
				copy_net.train()
				y_tmp = copy_net(x, age)
				loss_tmp = neg_par_log_likelihood(y_tmp, ytime, yevent)
				S_loss.append(loss_tmp)
			print(type(S_loss))
			print(type(S_set))
			S_loss = torch.tensor(S_loss)
			S_loss = S_loss.detach().numpy()
			S_set = S_set.detach().numpy()
			print(type(S_loss))
			print(type(S_set))
			###apply cubic interpolation
			interp_S_loss = interp1d(S_set, S_loss, kind='cubic')
			interp_S_set = torch.linspace(min(S_set), max(S_set), steps=100)
			interp_loss = interp_S_loss(interp_S_set)
			optimal_S = interp_S_set[np.argmin(interp_loss)]
			optimal_param_mask = s_mask(sparse_level = optimal_S.item(), param_matrix = copy_param, nonzero_param_1D = copy_param_1d, dtype = dtype)
			if "sc2" in name:
				final_optimal_param_mask = torch.where(do_m1_grad_mask == 0, torch.ones_like(do_m1_grad_mask), optimal_param_mask)
				optimal_transformed_param = net_sc2_weight.mul(final_optimal_param_mask)
			if "sc3" in name:
				final_optimal_param_mask = torch.where(do_m2_grad_mask == 0, torch.ones_like(do_m2_grad_mask), optimal_param_mask)
				optimal_transformed_param = net_sc3_weight.mul(final_optimal_param_mask)
			###update weights in copied net
			copy_state_dict[name].copy_(optimal_transformed_param)
			###update weights in net
			net_state_dict[name].copy_(optimal_transformed_param)

	###save the trained model
	torch.save(net.state_dict(), outpath)

	return

In [ ]:

#%%writefile Run_for_Interpret.py
# from DataLoader import load_data, load_pathway
# from Train_for_Interpret import InterpretCoxPASNet
# from Model import Cox_PASNet

import torch
import numpy as np


dtype = torch.FloatTensor
''' Net Settings'''
In_Nodes = 270 ###number of genes
Pathway_Nodes = 860 ###number of pathways
Hidden_Nodes = 100 ###number of hidden nodes
Out_Nodes = 30 ###number of hidden nodes in the last hidden layer
''' Initialize '''
Initial_Learning_Rate = 0.00075
L2_Lambda = 0.01
Num_EPOCHS = 900
###sub-network setup
Dropout_Rate = [0.7, 0.5]

''' load data and pathway '''
# pathway_mask = load_pathway("pathway_mask_tcga_ov.csv", dtype)
x, ytime, yevent, age = load_data("/content/entire_data_tcga_Brca.csv", dtype)

outpath = "InterpretCoxPASNet_lin_pred_brca_nopath.pt"
'''train Cox-PASNet for model interpretation'''
InterpretCoxPASNet(x, age, ytime, yevent, \
                   In_Nodes, Hidden_Nodes, Out_Nodes, \
					Initial_Learning_Rate, L2_Lambda, Num_EPOCHS, Dropout_Rate, outpath)

'''load trained Cox-PASNet'''
net = Cox_PASNet(In_Nodes, Hidden_Nodes, Out_Nodes)
net.load_state_dict(torch.load(outpath))
###if gpu is being used
if torch.cuda.is_available():
	net.cuda()
###
'''save weights and node values into files individually'''
w_sc1 = net.sc1.weight.data.cpu().detach().numpy()
w_sc2 = net.sc2.weight.data.cpu().detach().numpy()
w_sc3 = net.sc3.weight.data.cpu().detach().numpy()
w_sc4 = net.sc4.weight.data.cpu().detach().numpy()
np.savetxt("w_sc1_lin_pred_lung_nopath_150.csv", w_sc1, delimiter = ",")
np.savetxt("w_sc2_lin_pred_lung_nopath_150.csv", w_sc2, delimiter = ",")
np.savetxt("w_sc3_lin_pred_lung_nopath_150.csv", w_sc3, delimiter = ",")
np.savetxt("w_sc4_lin_pred_lung_nopath_150.csv", w_sc4, delimiter = ",")

pathway_node = net.tanh(net.sc1(x))
hidden_node = net.tanh(net.sc2(pathway_node))
hidden_2_node = net.tanh(net.sc3(hidden_node))
x_cat = torch.cat((hidden_2_node, age), 1)
lin_pred = net.sc4(x_cat)
np.savetxt("pathway_lin_pred_lung_nopath_150.csv", pathway_node.cpu().detach().numpy(), delimiter = ",")
np.savetxt("hidden_node_lin_pred_lung_nopath_150.csv", hidden_node.cpu().detach().numpy(), delimiter = ",")
np.savetxt("hidden_2_lin_pred_lung_nopath_150.csv", x_cat.cpu().detach().numpy(), delimiter = ",")
np.savetxt("lin_pred_lung_nopath_150.csv", lin_pred.cpu().detach().numpy(), delimiter = ",")


<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'torch.Tensor'>
<class 'numpy.nd

In [ ]:
np.savetxt("lin_pred_lung_nopath_150_.csv", lin_pred.cpu().detach().numpy(), delimiter = ",")